In [2]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(True,Filename)

In [3]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=9, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

In [4]:
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [ ]:
# n_pop = 10
# n_gen = 10
# n_sats = 5
# opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-8,-1))
# stk_object.dt = 30
# stk_object.Interpolate=True

# # Run optimization if read=True the 1st generation takes the input file as one of the initial population
# hof,percent,std,time = opt.run(read=True)

# print("Hall of Fame:")
# for entry in hof:
#     print(entry,"->",entry.fitness)

# # Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

# best_stats = opt.cost_function(hof[0])

In [ ]:
stk_object.dt = 2.5
stk_object.Generate_Holding_Data()

In [ ]:
stk_object.Plan(slew_rate=1)

In [ ]:
target_times = []
with alive_bar(324*len(stk_object.targets),force_tty=True,bar='classic',title=f'- Timing Planned Data',length=10) as bar:
    for tar_num in range(len(stk_object.targets)):
        times = []
        for bin_num in range(324):
            tar_window = stk_object.Holding_Data['Target'].values==tar_num
            bin_window = stk_object.Holding_Data['Bin Number'].values==bin_num
            times.append(min(stk_object.Holding_Data['Time'].values[tar_window&bin_window]))
            bar()
        target_times.append(max(times))

data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = np.array(target_times)/86400
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

data_comparison['Unplanned (Time)'].plot()
data_comparison['Planned (Time)'].plot()

In [ ]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()